In [2]:
import pandas_datareader as web
import datetime
import numpy as np
import pandas as pd
from time import sleep

In [4]:
tickers = pd.read_csv(r'{}.csv'.format("stock_list"), engine='python')
tickers.to_csv(r'{}.csv'.format("stock_list4"))
START_YR = "2009"
YRS_MIN = 10
RAW_DIR = "data/raw"
YRS_DIR = "data/yearly"
THREE_M_DIR = "data/3_months"
FORCE_UPDATE = False
REPASS = False
STOCK_LIST_DIR = "stock_list4"
START_LIST_NUM = 0 #(500/4)*3
END_LIST_NUM = 0

def set_col(df, c_name, c_type="str", tmp_i="False", v=np.nan):
    if not (c_name in df.columns):
        if c_type == "str":
            df[c_name] = ""
            df[c_name] = df[c_name].astype(str)
        else:
            df[c_name] = np.nan
    if not tmp_i == "False":
        if c_type == "str":
            try:
                df.at[tmp_i, c_name] = ""
            except:
                df[c_name] = ""
                
        df.at[tmp_i, c_name] = v
        

try:
    tickers = pd.read_csv(r'{}.csv'.format(STOCK_LIST_DIR), engine='python')
    tickers = tickers.loc[:, ~tickers.columns.str.contains('^Unnamed')]
except:
    pass
    tickers = pd.DataFrame()


set_col(tickers, 'symbol')
set_col(tickers, 'company')
set_col(tickers, 'price', "int")
set_col(tickers, 'ignore')
set_col(tickers, 'start_yr')
set_col(tickers, 'ignore_error')
set_col(tickers, 'adj_up_start_schedule')
set_col(tickers, 'adj_up_end_schedule')

if tickers.empty:
    tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))


for i, row in tickers.iterrows():
    if START_LIST_NUM>0 and i<START_LIST_NUM:
        continue
    if END_LIST_NUM>0 and END_LIST_NUM>START_LIST_NUM and i>END_LIST_NUM:
        break
    SYMBOL = row["symbol"]
#     SYMBOL = "PG"
    
    try:        
        current_stock = pd.read_csv(r'{}/{}_{}.csv'.format(RAW_DIR, SYMBOL, "history"), index_col=['Date'] , parse_dates=['Date'])
        tickers.at[i, "ignore"] = False
    except:
        current_stock = ""
        
    print(i+1, SYMBOL)
    if (pd.notnull(row['ignore']) and row['ignore'] and not REPASS):
        print("--Ignore: {}".format(row['ignore'] and not REPASS))
        continue
    if FORCE_UPDATE or not str(current_stock):
        print("--get data")
        try:
#             sleep(15)
            current_stock = web.DataReader(SYMBOL, data_source="yahoo", start=START_YR) 
            set_col(tickers, "ignore","str", i, False)
        except Exception as e:
            set_col(tickers, "ignore","str", i, True)
            set_col(tickers, "ignore_error","str", i, e)
            print("---Error: {}".format(e))
            tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))
            continue
    
        
#     for three months 70-75
    tickers.at[i, "price"] = current_stock.tail(1)["Close"][0]
    tickers.at[i, "ignore"] = skip = current_stock.tail(1).index[0].year-current_stock.head(1).index[0].year < YRS_MIN
    tickers.at[i, "start_yr"] = current_stock.head(1).index[0].year
    print("--Skip: {}".format(skip))
    if skip:
        current_stock.to_csv(r'{}/{}_{}.csv'.format("data/under_min", current_stock.head(1).index[0].year, SYMBOL))
        tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))
        continue
    else:
        current_stock.to_csv(r'{}/{}_{}.csv'.format(RAW_DIR, SYMBOL, "history"))

    yearly_adj_df = pd.DataFrame()
    yearly_df = pd.DataFrame()
    three_m_adj_df = pd.DataFrame()
    three_m_c_df = pd.DataFrame()
    three_m_pc_df = pd.DataFrame()
    for j, row2 in current_stock.iterrows():
        yearly_adj_df.at["{:02d}-{:02d}".format(j.month, j.day), j.year] = row2["Adj Close"]
        yearly_df.at["{:02d}-{:02d}".format(j.month, j.day), j.year] = row2["Close"]
  
    yearly_adj_df.sort_index(inplace=True)
    yearly_df.sort_index(inplace=True)
    for j, row2 in yearly_df.iterrows():
        adj_d_ups = 0
        adj_d_downs = 0
        c_d_ups = 0
        c_d_downs = 0
        for yr in yearly_df.columns:
            d = pd.to_datetime('{}-{}'.format(yr, j), format="%Y-%m-%d", dayfirst=True, errors='coerce')
            three_m_adj_df.at[j, yr] = 0
            three_m_c_df.at[j, yr] = 0
            three_m_pc_df.at[j, yr] = 0
            if not pd.isnull(d):
                f = d + datetime.timedelta(days=75)
                tmp_three_m_df = current_stock.loc['{}-{:02d}-{:02d}'.format(d.year, d.month, d.day):'{}-{:02d}-{:02d}'.format(f.year, f.month, f.day)]
                
                if not tmp_three_m_df.empty:
                    adj_first_3m_num = tmp_three_m_df.head(3)["Adj Close"].dropna().head(1)[0]
                    c_first_3m_num = tmp_three_m_df.head(3)["Close"].dropna().head(1)[0]              
                    adj_last_3m_num = tmp_three_m_df.tail(3)["Adj Close"].dropna().head(1)[0]
                    c_last_3m_num = tmp_three_m_df.tail(3)["Close"].dropna().head(1)[0]
                    
                    three_m_adj_df.at[j, yr] = adj_last_3m_num - adj_first_3m_num
                    three_m_c_df.at[j, yr] = c_last_3m_num - c_first_3m_num
                    three_m_pc_df.at[j, yr] = ((c_last_3m_num - c_first_3m_num)/c_last_3m_num)*100
                    if (adj_last_3m_num - adj_first_3m_num)>0:
                        adj_d_ups=adj_d_ups+1
                    else:
                        adj_d_downs=adj_d_downs+1

                    if (c_last_3m_num - c_first_3m_num)>0:
                        c_d_ups=c_d_ups+1
                    else:
                        c_d_downs=c_d_downs+1

                
        set_col(three_m_adj_df, "ups", "int", j, adj_d_ups)
        set_col(three_m_c_df, "ups", "int", j, c_d_ups)
        set_col(three_m_pc_df, "ups", "int", j, c_d_ups)
        set_col(tickers, "adj_ups", "int", i, adj_d_ups)
        set_col(tickers, "close_ups", "int", i, c_d_ups)
        set_col(three_m_adj_df, "downs", "int", j, adj_d_downs)
        set_col(three_m_c_df, "downs", "int", j, c_d_downs)
        set_col(three_m_pc_df, "downs", "int", j, c_d_downs)
        set_col(tickers, "adj_downs", "int", i, adj_d_downs)
        set_col(tickers, "close_downs", "int", i, c_d_downs)
        set_col(three_m_adj_df, "end_day_sum","str", j, '{:02d}-{:02d}'.format(f.month, f.day))
        set_col(three_m_c_df, "end_day_sum","str", j, '{:02d}-{:02d}'.format(f.month, f.day))
        
    three_m_adj_df['avg'] = three_m_adj_df.loc[:, ~three_m_adj_df.columns.str.contains('^ups|^downs|^avg|^end_day_sum', na=False)].mean(axis=1)
    three_m_c_df['avg'] = three_m_c_df.loc[:, ~three_m_c_df.columns.str.contains('^ups|^downs|^avg|^end_day_sum', na=False)].mean(axis=1)
    three_m_pc_df['avg'] = three_m_pc_df.loc[:, ~three_m_pc_df.columns.str.contains('^ups|^downs|^avg|^end_day_sum', na=False)].mean(axis=1)

    yearly_adj_df.to_csv(r'{}/{}_yrs_adj_data.csv'.format(YRS_DIR, SYMBOL))
    yearly_df.to_csv(r'{}/{}_yrs_close_data.csv'.format(YRS_DIR, SYMBOL))
    three_m_adj_df.sort_index(inplace=True)
    three_m_adj_df.to_csv(r'{}/{}_3m_adj_data.csv'.format(THREE_M_DIR, SYMBOL))
    three_m_c_df.sort_index(inplace=True)
    three_m_c_df.to_csv(r'{}/{}_3m_c_data.csv'.format(THREE_M_DIR, SYMBOL))
    three_m_pc_df.sort_index(inplace=True)
    three_m_pc_df.to_csv(r'{}/{}_3m_pc_data.csv'.format(THREE_M_DIR, SYMBOL))
    
    set_col(tickers, "adj_up_start_schedule","str", i, three_m_adj_df.sort_values(['ups', 'avg'], ascending=[False, False]).head(1).index[0])
    set_col(tickers, "adj_up_end_schedule","str", i, three_m_adj_df.sort_values(['ups', 'avg'], ascending=[False, False]).head(1)["end_day_sum"][0])
    set_col(tickers, "close_up_start_schedule","str", i, three_m_c_df.sort_values(['ups', 'avg'], ascending=[False, False]).head(1).index[0])
    set_col(tickers, "close_up_end_schedule","str", i, three_m_c_df.sort_values(['ups', 'avg'], ascending=[False, False]).head(1)["end_day_sum"][0])
    set_col(tickers, "close_up_percent_avg","int", i, three_m_pc_df.sort_values(['ups', 'avg'], ascending=[False, False]).head(1)["avg"][0])

    tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))
print("Done")

376 WDDMF
--get data
--Skip: True
377 BUDZ
--get data
--Skip: False
378 WEDG
--get data
---Error: No data fetched for symbol WEDG using YahooDailyReader
379 WCIG
--get data
--Skip: False
380 WECT
--get data
--Skip: False
381 WEC
--get data
--Skip: False
382 WZENY
--get data
---Error: No data fetched for symbol WZENY using YahooDailyReader
383 IPGOF
--get data
--Skip: True
384 IMPJY
--get data
--Skip: True
385 WEWWF
--get data
---Error: 'Date'
386 WEBEF
--get data
---Error: No data fetched for symbol WEBEF using YahooDailyReader
387 WBS
--get data
--Skip: False
388 WBSI
--get data
--Skip: False
389 WEJTY
--get data
--Skip: True
390 WEBJF
--get data
--Skip: True
391 WEBC
--get data
--Skip: False
392 MLBEF
--get data
---Error: No data fetched for symbol MLBEF using YahooDailyReader
393 WEB
--get data
--Skip: True
394 WEBB
--get data
--Skip: False
395 WFTLF
--get data
--Skip: True
396 WHSI
--get data
--Skip: False
397 WCCP
--get data
--Skip: True
398 HHHH
--get data
---Error: No data fetch